# Heuristic Search Approaches

If we have outside knowledge about our world, can we apply that knowledge to improve our search algorithms? When we add our own intelligence that's really the guts of third wave AI.

<p align="center">
<img src="images/3rd_wave_elsa.png">
</p>

## Greedy Best-first Search
So, for example, in the mapping problem in the text
suppose we also know (from another source) the point-to-point distances between the cities, independent of roads.
<p align="center">
<img src="images/Romania+with+step+costs+in+km.jpg">
</p>

So if I'm in Arad, there are three cities I can choose to go through as I try to find the best route to Bucharest. What's the best choice? What next?

It turns out that this greedy best-first algorithm will find a good path, but not quite the best path. But what did this algorithm ignore?

# A* Search

In the past algorithm, we informed the best-search algorithm with our external heuristic $f(n) = h(n)= $ the straight-line distance of city $n$ to Bucharest. But we completely ignored how far each city was from our starting city.

Now let's inform the best-search with 
\[
    f(n) = g(n) + h(n) 
\]
where $g(n)$ accounts for the travel distance to from the starting city to the frontier city.

## What does $h$ have to promise for A* to be cost optimal?

The heuristic has to be **admissible**, meaning that it *never overestimates* the distance to the goal.

A stronger condition is that a heuristic be **consistent**. Wikipedia gives a nicer explanation than the text:

Formally, for every node N and each successor P of N, the estimated cost of reaching the goal from N is no greater than the step cost of getting to P plus the estimated cost of reaching the goal from P. That is:

$$ h(N)\leq c(N,P)+h(P)$$ 

$$h(G)=0$$
where

* $h$ is the consistent heuristic function
* $N$ is any node in the graph
* $P$ is any descendant of $N$
* $G$ is any goal node
* $c(N,P)$ is the cost of reaching node $P$ from $N$
Informally, every node $i$ will give an estimate that, accounting for the cost to reach the next node, is always lesser than the estimate at node $i+1$. (This is the triangle inequality.)

Another term for a consistent heuristic is a **monotonic heuristic**.


<p align="center">
<img src="images/romania.jpg">
</p>
<p align="center">
<img src="images/astar_buch.jpg">
</p>


## How can A* work faster?

As it is, A* has to open all of its subnodes at each step to decide which is best. If two nodes have the same $f(n) = g(n) + h(n)$ then both have to be expanded. How can we adjust this? 

## Weighted A*

What if we decide being closer to the goal is more important than how far we've already traveled? 
* How far we've traveled is sunk cost at this point in the algorithm.
* What we want to do is save computation time and finish this thing.

So now states can expanded following the heuristic
$$ f(n) = g(n) + W \times h(n)$$
where $W > 1$. It may not give an optimal solution, but it will decide more quickly and the text calls this a **satisficing** 
solution.

<p align="center">
<img src="images/astar_v_weighted.png">
</p>


## Memory-bounded searches

Some quick tricks to reduce the burden on memory:
* for a larger problem, don't hold the frontier nodes as reached until done with them on the frontier (but it complicates code)
* Keep reference counts for states if we know how many visits we can have to that state before it's not going to offer an optional solution.
* **Beam search** only keep the $k$ best-performing nodes in the frontier and throw away the rest. *OR* only keep the nodes that are within $\delta$ of the best.
* **Iterative-deepening A* search (IDA*)** trades off holding visited nodes in memory with the cost of possibly re-visiting nodes that have already been searched. A cutoff-value is determined at each step and any nodes with $f$-values that are higher a discarded, creating a search contour. When the values are floats, this contour may consist of just one node at each step and perhaps too much information is lost.


Let's look at good slides from the University of Cambridge https://www.cl.cam.ac.uk/teaching/0809/ArtIntI/notes2.pdf

<p align="center">
<img src="images/ida0.png">
</p>
<p align="center">
<img src="images/ida1.png">
</p>
<p align="center">
<img src="images/ida2.png">
</p>
<p align="center">
<img src="images/ida3.png">
</p>


* **Recursive best-first search (RBFS)** starts to act like a recursive depth-first search, 


In [8]:
def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

In [9]:

def recursive_best_first_search(problem, h=None):
    """[Figure 3.26]"""
    # This caches h values on the nodes and can be
    # examined on the returned path
    h = memoize(h or problem.h, 'h')

    def RBFS(problem, node, flimit):
        # are you at your goal?
        if problem.goal_test(node.state):
            return node, 0  # (The second value is immaterial)
        # What's the frontier
        successors = node.expand(problem)
        # If I'm on a leaf and didn't solve the problem return horrible score
        # that allows us to look somewhere else
        if len(successors) == 0:
            return None, np.inf
        for s in successors:
            # each node's new cost is the path_cost to get there plus
            # the heuristic guess, or it's current f val, whichever is worse
            s.f = max(s.path_cost + h(s), node.f)
        while True:
            # Order by lowest f value
            successors.sort(key=lambda x: x.f)
            best = successors[0]
            # I can't search the successors if the best
            # is over the limit
            if best.f > flimit:
                return None, best.f
            # If there are choices record alt value
            if len(successors) > 1:
                alternative = successors[1].f
            else:
                alternative = np.inf
            # Here's where the cutoff is set
            result, best.f = RBFS(problem, best, min(flimit, alternative))
            if result is not None:
                return result, best.f

    node = Node(problem.initial)
    node.f = h(node)
    result, bestf = RBFS(problem, node, np.inf)
    return result

<p align="center">
<img src="images/rbfs0.png">
</p>
<p align="center">
<img src="images/rbfs1.png">
</p>
<p align="center">
<img src="images/rbfs2.png">
</p>
<p align="center">
<img src="images/rbfs3.png">
</p>
<p align="center">
<img src="images/rbfs4.png">
</p>
<p align="center">
<img src="images/rbfs5.png">
</p>
<p align="center">
<img src="images/rfbs_bucharest.jpg">
</p>


# Is this throwing the baby out with the bath water?
<img src="images/babu.png" width=300>

By remembering **nothing** we had to retrace steps. __MA*, memory bounded A*__ and it's friendly cousin __SMA*__ (Simplified...) expands until the memory is full. SMA* just drops the worst lef at this time and then backs up teh value of the forgotten node to its parent. So it's only going back to that subtree if it's desperate. Problems that require high memory resources can be theoretically solvable by A* but stuck in a _trashing_ pattern or having to repeatedly regenerate nodes.


# 3.6 Heuristic Functions

How important is the accuracy of your heuristic? Example: 8-puzzle



![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

The goal here is to get to

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz1.png)

So what kind of metrics could we assign the top-left puzzle?
* The **Hamming Distance** is the number of wrong tiles (8 for this guy, and 7 for the top-right).
* The **Manhattan Distance** is the sum of the distances of the tiles to get to the right place -- so are the tiles a little bit wrong or a lot wrong?

In [10]:
def manhattan(nodes, goals):
    X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
    Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
    return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                for (s, g) in zip(nodes, goals) if s != 0)

In [11]:
manhattan((5,2,7,8,4,0,1,3,6), (1,2,3,4,5,6,7,8,0))

14

In [12]:
2 + 0 + 4 + 2 + 1  + 2 + 3 + 1

15

I couldn't get the same answer for the example in the text, either, but we get the basic idea. The book discusses metrics that are good for performance comparisons. the *effective branching factor* and *effective depth* are determined determining how a uniform tree of depth $d$ would have to be constructed to reflect the number of nodes that were expanded in the search. But for us I think it's sufficient to note the number of nodes expanded by each approach.

| $d$ |    BFS  |   A*($h_1$) | A*($h_2$) |
| ---| -----| -------| -----|
| 2  |     5   |   6  |    6  |
| 4 |     33 |    12 |    12 | 2.06  1.49  1.49
| 6 |    128 |    24 |    19 | 2.01  1.42  1.34
| 8 |    368 |    48 |    31 | 1.91  1.40  1.30
|10 |   1033 |   116 |    48 | 1.85  1.43  1.27
|12 |   2672 |   279 |    84 | 1.80  1.45  1.28
|14 |   6783 |   678 |   174 | 1.77  1.47  1.31
|16 |  17270 |  1683 |   364 | 1.74  1.48  1.32
|18 |  41558 |  4102 |   751 | 1.72  1.49  1.34
|20 |  91493 |  9905 |  1318 | 1.69  1.50  1.34
|22 | 175921 | 22955 |  2548 | 1.66  1.50  1.34
|24 | 290082 | 53039 |  5733 | 1.62  1.50  1.36

When one heuristic is always better than another, we say that it (e.g. $h_2$) **dominates** the other (e.g. $h_1$)

## Relaxed problems

One way to generate heuristics is to change your problem to a more _relaxed_ problem -- one with fewer rules. 
* The Manhattan distance pretends you can just slam a tile into another tile
* The Romanian heuristic assumed you could just fly like a bird.

*ABSolver* (1990's) can find useful heuristics for this problem as well as rubics cube etc.

## Other ways to generate better heuristics

* Create **pattern databases** that store the exact solution costs for every possible subproblem. I personally think this is cheating. But it actually does let you solve the problem very quickly. 
* Clever ppl can combine disjoint puzzle databases to solve the 15-puzzle quickly.

* **Landmarks** are things that every plan for a task must eventually satisfy. You calculate the exact costs between each landmarks and the other nodes:
$$h_L(n) = \min_{L \in \texttt{Landmarks}} C^*(n,L) + C^*(L, \texttt{goal})$$
It turns out that this isn't quite admissible, although it is efficient. However, the associated **differential heuristic** is admissible:
$$ h_{DH}(n) = \max_{L\in\texttt{Landmarks}} | C^*(n,L) - C^*(\texttt{goal}, L)|$$
This works because the most useful landmarks are over-estimates. This difference gives us an estimate for the distance to the goal if any of the landmarks over-shoot our goal.

## But this is an AI book so can't we learn how to search better?

Yes. Each state in a **metalevel state space** captures the computational state of a program that's searching the original object-level state space. So maybe we'll get into this once we really learn reinforcement learning.

Also, we could take a bunch of randomly generated problem instances and collect statistics about solution costs. Then we could create a machine learning model that predicts costs given state information and use that as a heuristic.